In [60]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate


DATA_DIR = (os.getcwd() + "/data/")
start_date = '1999-01-01'
end_date = "2019-12-31"
# Get fundamental data for each stock in the ticker and append to the dataframe


def get_all_symbols():
    return [v.strip('.csv') for v in os.listdir(DATA_DIR)]


tickers = get_all_symbols()


In [62]:
ticker_data.head(10)


,Date,Open,High,Low,Close,Adj Close,Volume,Returns,Short Term Reversal,Stock Momentum,Long Term Reversal,Market_Beta,Turnover Volatility,Dividends,Total Returns,Total Return Volatility,Direction
0,1999-02-03,0.348214,0.362165,0.345982,0.358817,0.304563,338744000,1.025520,0.001558,0.001558,-0.072151,1.064066,4.259781e+08,0.0,0.025520,0.036932,-1
1,1999-02-04,0.358817,0.359375,0.337054,0.338170,0.287038,463780800,0.942458,-0.073393,-0.073393,-0.092814,1.243199,3.984316e+08,0.0,-0.057542,0.038231,-1
2,1999-02-05,0.341518,0.342634,0.316964,0.324219,0.275196,777201600,0.958746,-0.118362,-0.118362,-0.193056,1.299115,3.547813e+08,0.0,-0.041254,0.033808,1
3,1999-02-08,0.327567,0.338728,0.323661,0.337054,0.286090,468227200,1.039588,-0.077860,-0.077860,-0.161111,1.329342,3.556811e+08,0.0,0.039588,0.035544,-1
4,1999-02-09,0.338728,0.348772,0.330915,0.332031,0.281827,701153600,0.985097,-0.051036,-0.051036,-0.189373,1.237813,3.563124e+08,0.0,-0.014903,0.034968,1
5,1999-02-10,0.329241,0.345424,0.321429,0.342076,0.290353,563628800,1.030253,-0.046656,-0.046656,-0.169376,1.273495,3.527052e+08,0.0,0.030253,0.035964,1
6,1999-02-11,0.345982,0.354911,0.344308,0.353795,0.300300,565196800,1.034259,0.046205,0.046205,-0.147850,1.295339,3.359356e+08,0.0,0.034258,0.037061,-1
7,1999-02-12,0.349330,0.349330,0.330357,0.336496,0.285617,428904000,0.951104,0.037866,0.037866,-0.089123,1.374354,1.975405e+08,0.0,-0.048896,0.029954,1
8,1999-02-16,0.347098,0.347098,0.338170,0.342076,0.290353,300227200,1.016583,0.014900,0.014900,-0.072618,1.384519,1.628264e+08,0.0,0.016583,0.030311,-1
9,1999-02-17,0.340402,0.345424,0.329799,0.330357,0.280406,296060800,0.965742,-0.005042,-0.005042,-0.094801,1.414276,1.663125e+08,0.0,-0.034259,0.031062,-1


In [70]:
# Random Forest Classifier Model to predict the direction of the stock
ticker_data = pd.read_csv(DATA_DIR + 'AAPL.csv')
ticker_data['Direction'] = np.where(ticker_data['Close'].shift(-1) > ticker_data['Close'], 1, -1)

ticker_data.head(10)

# Split the data into train and test set
features = ticker_data[['Short Term Reversal', 'Adj Close', 'Volume',
                        'Long Term Reversal', 'Stock Momentum', 'Market_Beta', 'Turnover Volatility', 
                        'Returns', 'Total Return Volatility', 'Total Returns']]
target = ticker_data['Direction']
X = np.array(features).reshape(-1,10)  # input as columns
y = np.array(target)  # output as rows

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [10,20, 30, 100, None],
    'min_samples_leaf': [50, 75, 100, 150],
    'criterion': ['gini', 'entropy'],
    'max_features': ['auto', 'sqrt', 'log2']
}
model = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid, cv=10)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
# Print the cross-validation scores

# Make predictions on the testing data
grid_search_predictions = grid_search.predict(X_test)

# Calculate accuracy on the testing data
accuracy = accuracy_score(y_test, grid_search_predictions)
print("Accuracy on Test Data:", accuracy)

# Generate a classification report to assess precision, recall, and F1-score
print(classification_report(y_test, grid_search_predictions))



C:\Users\yinki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
1600 fits failed out of a total of 4800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1600 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\yinki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\yinki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-pac

{'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 100, 'n_estimators': 300}
Accuracy on Test Data: 0.5345155161494617
              precision    recall  f1-score   support

          -1       0.49      0.38      0.43       728
           1       0.56      0.67      0.61       851

    accuracy                           0.53      1579
   macro avg       0.53      0.52      0.52      1579
weighted avg       0.53      0.53      0.52      1579



In [71]:
tickers = ['AAPL']
for ticker in tickers:
    ticker_data = pd.read_csv(f"data/{ticker}.csv", index_col=0)
    ticker_data['Next Day Returns'] = ticker_data['Returns'].shift(-1)
    ticker_data.dropna(inplace=True)
    #print(ticker_data.head())
    features = ticker_data[['Short Term Reversal', 'Adj Close', 'Volume', 'Long Term Reversal', 'Stock Momentum', 'Market_Beta', 'Turnover Volatility', 
                            'Returns', 'Total Return Volatility', 'Total Returns']]
    target = ticker_data['Next Day Returns']
    
    X = np.array(features).reshape(-1, 10)  # input as columns
    y = np.array(target)  # output as rows
    
    
    
    feature_train, feature_test, target_train, target_test = train_test_split(X, y, test_size=0.3, random_state=0)
    #'''
    #Building RF model
    random_forest = RandomForestRegressor(n_jobs=-1, random_state=123, oob_score=True)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 15, 20, 30, None],
        'min_samples_leaf': [50, 75, 100],
        'criterion': ['absolute_error', 'squared_error', 'friedman_mse'],
        'max_features': [None, 'sqrt', 'log2'],
    }   
    
    grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=5)
    grid_search.fit(feature_train, target_train)
    
    print(grid_search.best_params_)
    
    criterion, max_depth, max_features, min_samples_leaf, n_estimators = grid_search.best_params_.values()
    
    #Building MLP model
    mlp = MLPRegressor(max_iter=1000)
    
    param_grid = {
        'hidden_layer_sizes': [(50, 50), (100, 50, 25), (100, 100)],
        'activation': ['relu', 'tanh', 'identity'],
        #'learning_rate': ['constant', 'adaptive'],
        'alpha': [0.0001, 0.001, 0.01],
    }

    # Create a GridSearchCV instance
    grid_search = GridSearchCV(mlp, param_grid, cv=5,
                           n_jobs=-1, scoring='neg_mean_squared_error')

    # Fit the GridSearchCV to find the best hyperparameters
    grid_search.fit(feature_train, target_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print(best_params)
    #'''
    #base_models = [
    #    ('rf', RandomForestRegressor(criterion='absolute_error', max_depth=10, max_features='log2', min_samples_leaf=100, n_estimators=200,
    #                                 oob_score=True, random_state=123, n_jobs=-1)),
    #    ('nn', MLPRegressor(max_iter=1000, activation='identity', alpha=0.001, hidden_layer_sizes=(100, 50, 25)))
    #]
    
    base_models = [
        ('rf', RandomForestRegressor(criterion=criterion, max_depth=max_depth, max_features=max_features, min_samples_leaf=min_samples_leaf, n_estimators=n_estimators,
                                     oob_score=True, random_state=123, n_jobs=-1)),
        ('nn', MLPRegressor(max_iter=1000, **best_params))
    ]
    
    meta_learner = GradientBoostingRegressor(random_state=42, n_estimators=300, learning_rate=0.05, max_depth=6, max_features='sqrt')
    #stack_model = StackingRegressor(estimators=base_models, final_estimator=meta_learner)
    #Grid Search for meta learner
    #param_grid = {
    #    'final_estimator__n_estimators': [100, 200, 300],
    #    'final_estimator__learning_rate': [0.01, 0.1, 0.2],
    #    'final_estimator__max_depth': [3, 4, 5]  # Example parameter choices
    #}

    #grid_search = GridSearchCV(estimator=stack_model, param_grid=param_grid,
    #                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    #grid_search.fit(feature_train, target_train)
    #print(grid_search.best_params_)
    
    #best_params = grid_search.best_params_
    #best_meta_learner = grid_search.best_estimator_

    
    final_stack_model = StackingRegressor(estimators=base_models, final_estimator=meta_learner)
    # Fit the stacking model on the training data
    final_stack_model.fit(feature_train, target_train)

    # Make predictions on the validation set
    y_pred = final_stack_model.predict(feature_test)
    
   
    #print(confusion_matrix(target_test, grid_search_predictions))
    print(f"Mean Absolute Error: {mean_absolute_error(target_test, y_pred)}")
    print(f"Mean Squared Error: {mean_squared_error(target_test, y_pred)}")
    print(f"R^2: {r2_score(target_test, y_pred)}")
    
    #Add the predictions to the dataframe only for the test data, avoid look ahead bias
    ticker_data['Next Day Returns Predictions'] = np.nan
    ticker_data.iloc[-len(y_pred):, ticker_data.columns.get_loc('Next Day Returns Predictions')] = y_pred
    ticker_data[['Next Day Returns', 'Next Day Returns Predictions']].plot(
        figsize=(15, 5))

    
    
    
    

In [69]:
ticker_data[['Next Day Returns', 'Next Day Returns Predictions']].tail(20)


,Next Day Returns,Next Day Returns Predictions
Date,,
2019-11-29,0.988438,0.999782
2019-12-02,0.982170,0.999432
2019-12-03,1.008826,1.000249
2019-12-04,1.014671,0.997803
2019-12-05,1.019316,1.006142
2019-12-06,0.986000,0.999136
2019-12-09,1.005844,1.002277
2019-12-10,1.008529,0.950992
2019-12-11,1.002548,1.000813


In [13]:
tickers = ['AAPL']
for ticker in tickers:
    ticker_data = pd.read_csv(f"data/{ticker}.csv", index_col=0)
    # print(ticker_data.head())
    features = ticker_data[['Short Term Reversal', 'Adj Close', 'Volume',
                            'Long Term Reversal', 'Stock Momentum', 'Market_Beta', 'Turnover Volatility']]
    target = ticker_data['Returns']
    
    X = np.array(features).reshape(-1, 7)  # input as columns
    y = np.array(target).reshape(-1, 1)  # output as rows
    
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    y = scaler.fit_transform(y)
    
    y = y.ravel()


    feature_train, feature_test, target_train, target_test = train_test_split(
        X, y, test_size=0.2, random_state=0)
    random_forest = RandomForestRegressor(
        n_jobs=-1, random_state=123, oob_score=True)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 15, 20, 30],
        'min_samples_leaf': [50, 75, 100],
        'criterion': ['absolute_error', 'squared_error', 'friedman_mse'],
        'max_features': [None, 'sqrt', 'log2'],
    }

    grid_search = GridSearchCV(
        estimator=random_forest, param_grid=param_grid, cv=10)
    grid_search.fit(feature_train, target_train)

    print(grid_search.best_params_)

    grid_search_predictions = grid_search.predict(feature_test)

    # print(confusion_matrix(target_test, grid_search_predictions))
    print(mean_absolute_error(target_test, grid_search_predictions))


KeyboardInterrupt: 

In [43]:
print(target.iloc[int(len(target)* 0.8):])

Date
2016-01-01    0.924758
2016-02-01    0.993322
2016-03-01    1.133327
2016-04-01    0.860079
2016-05-01    1.065287
2016-06-01    0.963169
2016-07-01    1.090063
2016-08-01    1.018137
2016-09-01    1.071276
2016-10-01    1.004334
2016-11-01    0.973402
2016-12-01    1.053335
2017-01-01    1.047747
2017-02-01    1.128884
2017-03-01    1.053236
2017-04-01    0.999930
2017-05-01    1.063418
2017-06-01    0.946678
2017-07-01    1.032703
2017-08-01    1.102670
2017-09-01    0.943446
2017-10-01    1.096808
2017-11-01    1.016623
2017-12-01    0.988294
2018-01-01    0.989364
2018-02-01    1.063848
2018-03-01    0.945790
2018-04-01    0.984980
2018-05-01    1.130764
2018-06-01    0.994402
2018-07-01    1.027984
2018-08-01    1.196227
2018-09-01    0.995175
2018-10-01    0.969522
2018-11-01    0.815955
2018-12-01    0.886384
2019-01-01    1.055154
2019-02-01    1.040315
2019-03-01    1.101730
2019-04-01    1.056436
2019-05-01    0.872427
2019-06-01    1.134873
2019-07-01    1.076394
2019-0